In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from metadata import *

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

Setting environment for Local Macbook Pro


In [2]:
stack = 'MD592'

In [3]:
outerContour_volume = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_down32_outerContourVolume.bp' % {'stack':stack})).astype(np.int8)

# with open(os.path.join(volume_dir, stack, stack+'_down32_outerContourVolume_bbox.txt'), 'r') as f:
#     oc_xmin, oc_xmax, oc_ymin, oc_ymax, oc_zmin, oc_zmax = map(int, f.readline().strip().split())

oc_xmin, oc_xmax, oc_ymin, oc_ymax, oc_zmin, oc_zmax = DataManager.load_volume_bbox(stack=stack, type='shell', downscale=32)
origin = (oc_xmin, oc_ymin, oc_zmin)

In [4]:
t = time.time()

# Use Meshlab to 1. compute normal at every point; 2. Poisson reconstruction
# http://www.andrewhazelden.com/blog/2012/04/automate-your-meshlab-workflow-with-mlx-filter-scripts/

vertices = np.array(np.where(outerContour_volume)).T[:, [1,0,2]] + origin
print 'Total vertices', len(vertices)
vertices = vertices[np.random.choice(np.arange(len(vertices)), 10000, replace=False)]

outer_sparse_polydata = mesh_to_polydata(vertices, [])
# save_mesh(polydata, os.path.join(MESH_ROOTDIR, stack, "%(stack)s_down32_outerContourVolume_sparse.ply" % {'stack': stack}))

# poisson_reconstruct_meshlab(input_fn=os.path.join(MESH_ROOTDIR, stack, "%(stack)s_down32_outerContourVolume_sparse.ply" % {'stack': stack}),
#                            output_fn=os.path.join(MESH_ROOTDIR, stack, "%(stack)s_down32_outerContourVolume_smoothed.stl" % {'stack': stack}))

outer_smooth_polydata = poisson_reconstruct_meshlab(polydata=outer_sparse_polydata)

mesh_fn = os.path.join(MESH_ROOTDIR, stack, "%(stack)s_down32_outerContourVolume_smoothed.stl" % {'stack': stack})

save_mesh(outer_smooth_polydata, mesh_fn)

# VTK SurfaceReconstructFilter - likely program bug, distance function is wrong outside the contour.
# polydata = vertices_to_surface(vertices, num_simplify_iter=0, smooth=True)

# VTK marching cube - if applied to points rather than volume, results in separate vertices, not contiguous surface.
# polydata = volume_to_polydata(outerContour_volume, origin=origin, num_simplify_iter=1, smooth=True)

# save_mesh_stl(polydata, os.path.join(MESH_ROOTDIR, stack, "%(stack)s_down32_outerContourVolume_smoothed.stl" % {'stack': stack}))

sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5

Total vertices 319294
meshlabserver -i /tmp/input.ply -o /tmp/output.stl -s /home/yuncong/Brain/3d/outerContour_poisson_reconstruct.mlx -om vc vn


mesh_to_polydata: No faces are provided, so skip simplification.
Child returned 0
time: 3.75


In [5]:
# Load

In [6]:
mesh_fn = os.path.join(MESH_ROOTDIR, stack, "%(stack)s_down32_outerContourVolume_smoothed.stl" % {'stack': stack})

outerContour_polydata = load_mesh_stl(mesh_fn, return_polydata_only=True)

outerContour_actor = actor_mesh(outerContour_polydata, (1,1,1), opacity=.1, wireframe=False)

fill point array: 0.01 seconds
fill cell array: 0.00 seconds


In [7]:
launch_vtk([outerContour_actor], init_angle='30', background_color=(0,0,0))